In [283]:
import numpy as np
import random
import math
from typing import Literal

In [284]:
#Types
Tile = str
Coordinates = tuple[int, int]
Up = tuple[Literal[-1], Literal[0]]
Down = tuple[Literal[1], Literal[0]]
Left = tuple[Literal[0], Literal[-1]]
Right = tuple[Literal[0], Literal[1]]
Direction = Up | Down | Left | Right
Compatibility = tuple[Tile, Tile, Direction]
Weights = dict[Tile, int]

UP = (-1, 0)
DOWN = (1, 0)
LEFT = (0, -1)
RIGHT = (0, 1)
DIRS = [UP, DOWN, LEFT, RIGHT]

# What we will do in this notebook ?
- Parse an input data to create rulesets for tiles
- Apply WFC to create a simple tiled version of WFC

In [285]:
class Cell:

    def __init__(self, pos, options: set[Tile]):
        self.position = pos # position is given as a (y, x) tuple
        self.options = options # list of possible options available
        self.collapsed = False # collapsed state of the cell
        self.value = None # chosen option for this cell

    def __repr__(self):
        return f'tile {self.position} \n' \
               f'options : {[i for i in self.options]} \n' \
               f'value : {self.value} \n' \
               f'collapsed : {self.collapsed} \n'

    def collapse(self):
        pass

    def updateOptions(self):
        pass

    def __le__(self, other):
        return self.entropy() < other.entropy()

    def __eq__(self, other):
        return self.entropy() == other.entropy()

    def __gt__(self, other):
        return self.entropy() > other.entropy()


# Parsing
- Looking at every possible combinations in a NxN grid
- Returns a list of possible rules

Rules are represented by a three element tuple : (elem1, elem2, direction)
Each rule should come with its counterpart from the other side

In [286]:
input_matrix = np.array([
    ['L','L','L','L'],
    ['L','L','L','L'],
    ['L','L','L','L'],
    ['L','C','C','L'],
    ['C','S','S','C'],
    ['S','S','S','S'],
    ['S','S','S','S'],
], dtype=Tile)

In [287]:
def valid_dirs(cur_coords:Coordinates, matrix_size:tuple[int, int]) -> list[Direction]:
    """
    Returns the valid directions from `cur_co_ord` in a matrix
    of `matrix_size`. Ensures that we don't try to take step to the
    left when we are already on the left edge of the matrix.
    :param cur_coords: the current coords
    :param matrix_size: the size of the matrix
    :return: a list of possible directions
    """
    y, x = cur_coords
    dirs : list[Coordinates] = []
    height, width = matrix_size

    if y > 0: dirs.append(UP)
    if y < height - 1 : dirs.append(DOWN)
    if x > 0: dirs.append(LEFT)
    if x < width - 1 : dirs.append(RIGHT)

    return dirs


In [288]:
def parseExampleMatrix(matrix: np.ndarray) -> tuple[Weights, set[Compatibility]]:
    """
    Parses an example matrix. It extracts :
    1. Tile compatibilities
    2. The weight of each tile type

    :param matrix: a 2D matrix of tiles
    :return: A tuple :
        * A set of compatible tile combinations
        * A dict of weights for each tile type
    """
    compatibilities : set[Compatibility] = set()
    weights: Weights = {}

    for y, row in enumerate(matrix):
        for x, cur_tile in enumerate(row):
            if cur_tile not in weights:
                weights[cur_tile] = 0
            weights[cur_tile] += 1

            for d in valid_dirs((y, x), matrix.shape):
                other = matrix[y+d[0], x+d[1]]
                compatibilities.add((cur_tile, other, d)) # Tile1, Tile2, Direction relative

    return weights, compatibilities

In [289]:
parseExampleMatrix(input_matrix)

({'L': 14, 'C': 4, 'S': 10},
 {('C', 'C', (0, -1)),
  ('C', 'C', (0, 1)),
  ('C', 'L', (-1, 0)),
  ('C', 'L', (0, -1)),
  ('C', 'L', (0, 1)),
  ('C', 'S', (0, -1)),
  ('C', 'S', (0, 1)),
  ('C', 'S', (1, 0)),
  ('L', 'C', (0, -1)),
  ('L', 'C', (0, 1)),
  ('L', 'C', (1, 0)),
  ('L', 'L', (-1, 0)),
  ('L', 'L', (0, -1)),
  ('L', 'L', (0, 1)),
  ('L', 'L', (1, 0)),
  ('S', 'C', (-1, 0)),
  ('S', 'C', (0, -1)),
  ('S', 'C', (0, 1)),
  ('S', 'S', (-1, 0)),
  ('S', 'S', (0, -1)),
  ('S', 'S', (0, 1)),
  ('S', 'S', (1, 0))})

# WaveFunction class
This class is responsible for handling the output grid

In [290]:
class WaveFunction:

    def __init__(self, size:tuple[int, int], options, weights):
        self.grid = np.empty(shape = size, dtype=Cell)
        self.size = size
        self.weights: Weights = weights
        for _y in range(size[0]):
            for _x in range(size[1]):
                self.grid[_y, _x] = Cell((_y, _x), options)

    def __repr__(self):
        display = f""
        for _y in range(self.size[0]):
            for _x in range(self.size[1]):
                display += f" {self.grid[_y, _x].value} "
            display += "\n"

        return display

    def getEntropyAtCoords(self, coords:Coordinates):
        weights_sum = sum([self.weights[w] for w in self.grid[coords].options])
        weights_log_sum = sum([self.weights[w] +
                               math.log(self.weights[w])
                               for w in self.grid[coords].options])
        return math.log(weights_sum) - (weights_log_sum / weights_sum)

    def getMinEntropyCel(self):
        min_entropy_coords: Coordinates = (0,0)
        min_entropy = None
        for y, row in enumerate(self.grid):
            for x, cell in enumerate(row):
                if self.grid[y, x].collapsed:
                    continue
                entropy = self.getEntropyAtCoords((y, x))
                # Add some noise
                entropy += (random.random() / 1000)
                if min_entropy is None or entropy < min_entropy :
                    min_entropy = entropy
                    min_entropy_coords = (y, x)

        return min_entropy_coords


    def propagate(self):
        pass

    def getStateOfGrid(self):
        pass


In [291]:
weights, comp = parseExampleMatrix(input_matrix)
options = ['L', 'C', 'S']

In [292]:
wfc = WaveFunction((3, 3), options, weights)

In [293]:
wfc.getEntropyAtCoords((0,2))

2.106206767899161

In [294]:
wfc.getMinEntropyCel()

(0, 2)